In [151]:
import pandas as pd
import numpy as np

In [251]:
grammyAwards = pd.read_csv("data/grammySongs_1999-2019.csv")
grammyAwards = grammyAwards[['Name', 'Artist', 'GrammyAward', 'GrammyYear']]
# grammyAwards = grammyAwards[['Name', 'Artist', 'GrammyAward']]
grammyAwards = grammyAwards.groupby(['Name', 'Artist', 'GrammyYear']).count().reset_index()
grammyAwards = grammyAwards.applymap(lambda s: s.upper() if type(s) == str else s)

# namesOnly = grammyAwards.groupby(['Name', 'Artist'])
# grammyAwards = namesOnly['Genre'].agg(lambda x: "".join(x)).reset_index(name="Genre")
# grammyAwards.head(50)

In [263]:
import difflib 
grammyAwards['Artist'] = grammyAwards['Artist'].map(lambda x: difflib.get_close_matches(x, billboardHot100.index))
billboardHot100.join(grammyAwards)


TypeError: object of type 'int' has no len()

In [262]:
%pip install fuzzywuzzy
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
fuzzy_merge(billboardHot100, grammyAwards, 'Artist', 'Artist', threshold=80)

Note: you may need to restart the kernel to use updated packages.


AttributeError: module 'fuzzywuzzy.fuzz' has no attribute 'fuzzy_merge'

In [183]:
original100 = pd.read_csv('data/billboardHot100_1999-2019.csv')
# print(original100[original100['Name'] == "Bye Bye Bye"])

In [252]:
billboardHot100 = pd.read_csv('data/billboardHot100_1999-2019.csv')
billboardHot100 = billboardHot100[['Artists', 'Name', 'Genre', 'Peak.position', 'Weeks.on.chart','Weekly.rank','Date']]
billboardHot100['Weeks.on.chart'] = billboardHot100['Weeks.on.chart'].fillna(1)
billboardHot100['Peak.position'] = billboardHot100['Peak.position'].fillna(100)
billboardHot100['Weekly.rank'] = billboardHot100['Weekly.rank'].fillna(100)
billboardHot100['Genre'] = billboardHot100['Genre'].fillna('None')
billboardHot100['Date'] = billboardHot100['Date'].fillna('1999')
# billboardHot100.groupby(by=['Artist', 'Name']).min('Peak.position').max('Weeks.on.chart')
billboardHot100 = billboardHot100.groupby(['Artists', 'Name','Genre']).agg({'Peak.position':'min', 'Weeks.on.chart':'max', 'Weekly.rank':'mean', 'Date':'max'})[['Peak.position','Weeks.on.chart','Weekly.rank', 'Date']].reset_index()
billboardHot100.rename(columns = {'Weekly.rank':'Avg.rank'}, inplace = True)
billboardHot100.rename(
    columns={"Artists":"Artist"}, inplace=True)
billboardHot100['Date'] = pd.to_datetime(billboardHot100['Date'])
billboardHot100['Date'] = billboardHot100['Date'].apply(lambda x : x.year)
billboardHot100 = billboardHot100.applymap(lambda s: s.upper() if type(s) == str else s)
billboardHot100.head(5)


,Artist,Name,Genre,Peak.position,Weeks.on.chart,Avg.rank,Date
0,"""WEIRD AL"" YANKOVIC",CANADIAN IDIOT,"COMEDY,PARODY,ROCK",82.0,3.0,91.333333,2006
1,"""WEIRD AL"" YANKOVIC",WORD CRIMES,"PARODY,COMEDY,POP",39.0,2.0,65.000000,2014
2,'N SYNC,BYE BYE BYE,"ADULT CONTEMPORARY,DANCE-POP,TEEN POP,BOY BAND...",4.0,23.0,14.260870,2000
3,'N SYNC,GONE,"ADULT CONTEMPORARY,ACOUSTIC,SOUL POP,R&;B,BOY ...",11.0,24.0,27.125000,2001
4,'N SYNC,IT'S GONNA BE ME,"MEMES,DANCE-POP,TEEN POP,BOY BAND,POP",1.0,25.0,21.680000,2000


In [154]:
from sklearn.preprocessing import MultiLabelBinarizer

genres_as_list = [g.split(",") for g in billboardHot100['Genre']]
one_hot = MultiLabelBinarizer()
genre_encoding = one_hot.fit_transform(genres_as_list)
genres = one_hot.classes_

genre_df = pd.DataFrame(genre_encoding, columns = genres)
# genre_df.head()
billboardHot100 = billboardHot100.join(genre_df)
billboardHot100 = billboardHot100.drop(columns=['Genre'])
billboardHot100

,Artist,Name,Peak.position,Weeks.on.chart,Avg.rank,A CAPPELLA,ACOUSTIC,ADULT ALTERNATIVE,ADULT CONTEMPORARY,AFRICAN LANGUAGES,...,TROPICAL HOUSE,TV,UK,UK RAP,UNDERGROUND HIP-HOP,WEST COAST,WORLD MUSIC,WORSHIP,YOUTUBE,\XD0\XA0\XD0\XBE\XD1\X81\XD1\X81\XD0\XB8\XD1\X8F (RUSSIA)
0,"""WEIRD AL"" YANKOVIC",CANADIAN IDIOT,82.0,3.0,91.333333,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"""WEIRD AL"" YANKOVIC",WORD CRIMES,39.0,2.0,65.000000,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,'N SYNC,BYE BYE BYE,4.0,23.0,14.260870,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,'N SYNC,GONE,11.0,24.0,27.125000,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,'N SYNC,IT'S GONNA BE ME,1.0,25.0,21.680000,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7209,WILL.I.AM,FALL DOWN,100.0,1.0,58.000000,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7210,WILL.I.AM,I GOT IT FROM MY MAMA,31.0,9.0,56.666667,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7211,WILL.I.AM,IT'S A NEW DAY,78.0,2.0,81.000000,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7212,"WILL.I.AM, BRITNEY SPEARS",SCREAM & SHOUT,3.0,24.0,14.583333,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [248]:
from sklearn.preprocessing import MultiLabelBinarizer

genres_as_list = [g.split(",") for g in billboardHot100['Genre']]
one_hot = MultiLabelBinarizer()
genre_encoding = one_hot.fit_transform(genres_as_list)
genres = one_hot.classes_

# genre_df = pd.DataFrame(genres, columns=['genre'])
# genre_df.to_csv('genreList.csv', index=False)
print(genres)

['A CAPPELLA' 'ACOUSTIC' 'ADULT ALTERNATIVE' 'ADULT CONTEMPORARY'
 'AFRICAN LANGUAGES' 'AFRO TRAP' 'AFROBEATS' 'ALTERNATIVE'
 'ALTERNATIVE COUNTRY' 'ALTERNATIVE DANCE' 'ALTERNATIVE METAL'
 'ALTERNATIVE POP' 'ALTERNATIVE R&;B' 'ALTERNATIVE ROCK' 'AMBIENT'
 'AMERICAN IDOL' 'AMERICAN UNDERGROUND' 'AMERICANA' 'ANIME' 'ART POP'
 'ART ROCK' 'ATLANTA' 'AUSSIE HIP-HOP' 'AUSTRALIA' 'AVANT GARDE' 'BACHATA'
 'BALLAD' 'BAROQUE POP' 'BASKETBALL' 'BASS MUSIC' 'BASSLINE' 'BAY AREA'
 'BEDROOM POP' 'BEEF' 'BIG BAND' 'BLUE-EYED SOUL' 'BLUEGRASS' 'BLUES'
 'BLUES ROCK' 'BOLIVIA' 'BOOM BAP' 'BOSSA NOVA' 'BOUNCE' 'BOY BAND'
 'BRASIL' 'BRIT POP' 'BRITISH ROCK' 'BROADWAY' 'BUBBLEGUM POP' 'CALYPSO'
 'CANADA' 'CELTIC' 'CHAMBER MUSIC' 'CHARITY' 'CHART HISTORY'
 'CHICAGO DRILL' 'CHILDREN&#39;S MUSIC' 'CHILL' 'CHILLHOP' 'CHILLSTEP'
 'CHRISTIAN' 'CHRISTIAN METAL' 'CHRISTIAN POP' 'CHRISTIAN ROCK'
 'CHRISTMAS' 'CIVIL RIGHTS' 'CLASSICAL CROSSOVER' 'CLASSICAL MUSIC'
 'CLIMATE CHANGE' 'CLOUD RAP' 'COLOMBIA' 'COMEDY' 'CO

In [155]:
songAttributes = pd.read_csv('data/songAttributes_1999-2019.csv')
songAttributes = songAttributes[['Name', 'Artist', 'Energy', 'Danceability', 'Tempo', 'Valence']]
songAttributes = songAttributes.applymap(lambda s: s.upper() if type(s) == str else s)
songAttributes.head(5)

,Name,Artist,Energy,Danceability,Tempo,Valence
0,WELCOME ALL AGAIN,COLLECTIVE SOUL,0.904,0.520,106.022,0.365
1,FUZZY,COLLECTIVE SOUL,0.709,0.581,120.027,0.408
2,DIG,COLLECTIVE SOUL,0.918,0.572,144.061,0.370
3,YOU,COLLECTIVE SOUL,0.661,0.596,111.975,0.183
4,MY DAYS,COLLECTIVE SOUL,0.808,0.520,92.721,0.666


In [218]:
inProgress = grammyAwards.merge(songAttributes, on=['Name', 'Artist'], how='left').drop_duplicates(subset=['Name', 'Artist'], keep='first')
# inProgress.rename(columns={"GrammyYear":"Date"}, inplace=True)
inProgress.head()

,Name,Artist,GrammyYear,GrammyAward,Energy,Danceability,Tempo,Valence
0,'TIL SUMMER COMES AROUND,KEITH URBAN,2010,1,0.629,0.570,127.907,0.308
1,24 FRAMES,"ASON ISBELL, SONGWRITER",2015,1,NaN,NaN,NaN,NaN
2,24K MAGIC,BRUNO MARS,2017,1,0.803,0.818,106.970,0.632
3,500 MILES HIGH,"CHICK COREA, SOLOIST",2011,1,NaN,NaN,NaN,NaN
4,99 PROBLEMS,JAY-Z,2004,1,NaN,NaN,NaN,NaN


In [219]:
combined_df = inProgress.merge(billboardHot100, on=['Name', 'Artist'], how='left')
combined_df.head(50)

,Name,Artist,GrammyYear,GrammyAward,Energy,Danceability,Tempo,Valence,Genre,Peak.position,Weeks.on.chart,Avg.rank,Date
0,'TIL SUMMER COMES AROUND,KEITH URBAN,2010,1,0.629,0.570,127.907,0.3080,NaN,NaN,NaN,NaN,NaN
1,24 FRAMES,"ASON ISBELL, SONGWRITER",2015,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,24K MAGIC,BRUNO MARS,2017,1,0.803,0.818,106.970,0.6320,"FUNK-POP,FUNK,POP",4.0,41.0,21.439024,2016.0
3,500 MILES HIGH,"CHICK COREA, SOLOIST",2011,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,99 PROBLEMS,JAY-Z,2004,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,ADORN,MIGUEL,2012,1,0.576,0.625,179.063,0.2350,"POP,NEO SOUL,SINGER-SONGWRITER,TRIP-HOP,SOUL,S...",17.0,33.0,36.060606,2012.0
6,AGAIN,LENNY KRAVITZ,2000,1,NaN,NaN,NaN,NaN,"SOUL,PRODUCER,SINGER-SONGWRITER,ALTERNATIVE RO...",4.0,32.0,18.750000,2000.0
7,AIN'T NO OTHER MAN,CHRISTINA AGUILERA,2006,1,0.742,0.862,127.910,0.5110,"R&;B,FUNK,POP",6.0,20.0,19.555556,2006.0
8,AIN'T IT FUN,PARAMORE,2014,1,0.893,0.610,104.031,0.6730,"ADULT ALTERNATIVE,ADULT CONTEMPORARY,ALTERNATI...",10.0,24.0,29.958333,2013.0
9,ALRIGHT,"ENDRICK DUCKWORTH, KAWAN PRATHER, MARK ANTHONY...",2015,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [158]:
%pip install spotipy

Note: you may need to restart the kernel to use updated packages.


In [159]:
import spotipy
import spotipy.oauth2 as oauth2
from spotipy.oauth2 import SpotifyOAuth
from spotipy.oauth2 import SpotifyClientCredentials
import time

In [244]:
auth_manager = SpotifyClientCredentials(client_id = '24e3fb8d86594f38b3481d1f0f26316b', 
                                        client_secret = '1ccafaf87d5b413d86b87e06b76c7386')
sp = spotipy.Spotify(auth_manager=auth_manager)
artist_info = 'spotify:artist:3mIj9lX2MWuHmhNCA7LSCW'

artist = sp.artist(artist_info)
print(artist['genres'])
artist

['modern alternative rock', 'modern rock', 'pop', 'rock']


{'external_urls': {'spotify': 'https://open.spotify.com/artist/3mIj9lX2MWuHmhNCA7LSCW'},
 'followers': {'href': None, 'total': 5406666},
 'genres': ['modern alternative rock', 'modern rock', 'pop', 'rock'],
 'href': 'https://api.spotify.com/v1/artists/3mIj9lX2MWuHmhNCA7LSCW',
 'id': '3mIj9lX2MWuHmhNCA7LSCW',
 'images': [{'height': 640,
   'url': 'https://i.scdn.co/image/ab6761610000e5eb592231ad18aab7a47772a958',
   'width': 640},
  {'height': 320,
   'url': 'https://i.scdn.co/image/ab67616100005174592231ad18aab7a47772a958',
   'width': 320},
  {'height': 160,
   'url': 'https://i.scdn.co/image/ab6761610000f178592231ad18aab7a47772a958',
   'width': 160}],
 'name': 'The 1975',
 'popularity': 78,
 'type': 'artist',
 'uri': 'spotify:artist:3mIj9lX2MWuHmhNCA7LSCW'}

In [245]:
artistName = 'JAY-Z'
trackName = '99 PROBLEMS'

# q = String.init(format:"artist:%@ track:%@",artistName,trackName)
song = sp.search(q=f"track:{trackName} year:2004", type="track", limit=1)
print(song['tracks']['items'][0]['artists'][0]['uri'])
# print(song)
track_id = song['tracks']['items'][0]['uri']
# print(track_id)
# song_info = sp.audio_analysis(track_id)
# print(song_info)
# track = sp.get_track(track_id)
# print(track)
# song_info[0].get('danceability')

spotify:artist:3nFkdlSjzX9mRTtwJOzDYB


In [220]:
combined_df.head()

,Name,Artist,GrammyYear,GrammyAward,Energy,Danceability,Tempo,Valence,Genre,Peak.position,Weeks.on.chart,Avg.rank,Date
0,'TIL SUMMER COMES AROUND,KEITH URBAN,2010,1,0.629,0.570,127.907,0.308,NaN,NaN,NaN,NaN,NaN
1,24 FRAMES,"ASON ISBELL, SONGWRITER",2015,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,24K MAGIC,BRUNO MARS,2017,1,0.803,0.818,106.970,0.632,"FUNK-POP,FUNK,POP",4.0,41.0,21.439024,2016.0
3,500 MILES HIGH,"CHICK COREA, SOLOIST",2011,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,99 PROBLEMS,JAY-Z,2004,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [225]:
for index, row in combined_df.head(50).iterrows():
    artistName = row['Artist']
    trackName = row['Name']
    year = row['GrammyYear']
#     print(artistName,trackName)
    try: 
        song = sp.search(q=f"track:{trackName}, year:{year}", type="track", limit=10)
        artist = song['tracks']['items'][0]['artists'][0]['name']
    #     print(artist)
        combined_df.at[index, 'Artist'] = artist
        track_id = song['tracks']['items'][0]['uri']
        song_info = sp.audio_features(track_id)
    #     print(song)
    #     print(song['tracks']['items'][0]['uri'])
           
        combined_df.at[index, 'Energy'] = song_info[0].get('energy')
        combined_df.at[index, 'Danceability'] = song_info[0].get('danceability')
        combined_df.at[index, 'Tempo'] = song_info[0].get('tempo')
        combined_df.at[index, 'Valence'] = song_info[0].get('valence')
    except:
        print("track not found", trackName)
    
combined_df.head(50)

track not found AIN'T NO OTHER MAN
track not found CITY OF BLINDING LIGHTS


,Name,Artist,GrammyYear,GrammyAward,Energy,Danceability,Tempo,Valence,Genre,Peak.position,Weeks.on.chart,Avg.rank,Date
0,'TIL SUMMER COMES AROUND,American Country Hits,2010,1,0.602,0.609,127.783,0.240,NaN,NaN,NaN,NaN,NaN
1,24 FRAMES,Jason Isbell,2015,1,0.527,0.568,104.930,0.761,NaN,NaN,NaN,NaN,NaN
2,24K MAGIC,Bruno Mars,2017,1,0.708,0.763,106.175,0.566,"FUNK-POP,FUNK,POP",4.0,41.0,21.439024,2016.0
3,500 MILES HIGH,Richard Franklin,2011,1,0.281,0.594,79.985,0.469,NaN,NaN,NaN,NaN,NaN
4,99 PROBLEMS,JAY-Z,2004,1,0.951,0.547,95.031,0.510,NaN,NaN,NaN,NaN,NaN
5,ADORN,Miguel,2012,1,0.576,0.625,179.063,0.235,"POP,NEO SOUL,SINGER-SONGWRITER,TRIP-HOP,SOUL,S...",17.0,33.0,36.060606,2012.0
6,AGAIN,Britney Spears,2000,1,0.834,0.751,95.053,0.894,"SOUL,PRODUCER,SINGER-SONGWRITER,ALTERNATIVE RO...",4.0,32.0,18.750000,2000.0
7,AIN'T NO OTHER MAN,CHRISTINA AGUILERA,2006,1,0.742,0.862,127.910,0.511,"R&;B,FUNK,POP",6.0,20.0,19.555556,2006.0
8,AIN'T IT FUN,ATC,2014,1,0.815,0.534,103.911,0.670,"ADULT ALTERNATIVE,ADULT CONTEMPORARY,ALTERNATI...",10.0,24.0,29.958333,2013.0
9,ALRIGHT,Kendrick Lamar,2015,1,0.766,0.796,110.034,0.558,NaN,NaN,NaN,NaN,NaN


In [164]:
print(len(combined_df))

366


In [240]:
song = sp.search(q=f"track:Mr. Saxobeat, year:2010", type="track", limit=10)
artist = len(song['tracks']['items'])#['items'][0]['artists'][0]['name']
print(artist)

0
